In [1]:
from utils import *
from sklearn.cross_validation import train_test_split
import time

/home/husein/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [657, 3117, 2165, 25, 4373, 352, 635, 10158, 366, 103] ['touch', 'haneke', 'challenges', 'us', 'confront', 'reality', 'sexual', 'aberration', 'remains', 'seen']


In [4]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [5]:
import tensorflow as tf

In [6]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta


def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size, activation):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate,
                          activation = activation)
    x = x[:, :-pad_sz, :]
    return x

class Model:
    def __init__(self, dict_size, dimension_output, size_layer, num_layers, kernel_size = 7):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        self.training = tf.placeholder(tf.bool, None)
        x = self.X
        
        with tf.variable_scope('embed'):
            x = embed_seq(x, dict_size, size_layer, 'word')
        x += position_encoding(x)
        
        def residual_block(x, size, rate, block):
            with tf.variable_scope('block_%d_%d' % (block, rate), reuse = False):
                pad_sz = (size - 1) * rate
                with tf.variable_scope('block_tanh'):
                    conv_filter = cnn_block(x, rate, pad_sz, x.shape[2], size, tf.nn.tanh)
                with tf.variable_scope('block_sigmoid'):
                    conv_gate = cnn_block(x, rate, pad_sz, x.shape[2], size, tf.nn.sigmoid)
                out = tf.multiply(conv_filter, conv_gate)
                out = tf.layers.dropout(out, 0.2, training=self.training) 
                with tf.variable_scope('block_out'):
                    out = layer_norm(out)
                    out = tf.layers.conv1d(
                        out,
                        size_layer,
                        kernel_size = 1,
                        strides = 1,
                        padding = 'same',
                        activation = tf.nn.tanh,
                    )
                return tf.add(x, out), out
            
        forward = tf.layers.conv1d(x, size_layer, kernel_size = 1, strides = 1, padding = 'SAME')
        zeros = tf.zeros_like(forward)
        for i in range(num_layers):
            for r in [1, 2, 4, 8, 16]:
                forward, s = residual_block(forward, size=kernel_size, rate=r, block=i)
                zeros = tf.add(zeros,s)
        zeros = layer_norm(zeros)
        self.logits = tf.layers.dense(zeros[:,-1], dimension_output)
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
maxlen = 500
size_layer = 128
num_layers = 2
learning_rate = 1e-4
batch_size = 32

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(len(dictionary),len(trainset.target_names),size_layer,num_layers)
sess.run(tf.global_variables_initializer())

In [9]:
vectors = str_idx(trainset.data,dictionary,maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(vectors, trainset.target,test_size = 0.2)

In [10]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.training: True
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.training: False
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.490702
time taken: 103.64606356620789
epoch: 0, training loss: 0.772093, training acc: 0.496183, valid loss: 0.712724, valid acc: 0.490702



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

time taken: 101.34961891174316
epoch: 1, training loss: 0.714211, training acc: 0.514501, valid loss: 0.709959, valid acc: 0.490702



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

...

time taken: 101.3210039138794
epoch: 11, training loss: 0.059577, training acc: 0.981123, valid loss: 1.856399, valid acc: 0.589222



train minibatch loop:   0%|          | 0/267 [00:00<?, ?it/s]

time taken: 101.2823498249054
epoch: 12, training loss: 0.039034, training acc: 0.987806, valid loss: 2.029441, valid acc: 0.590159



test minibatch loop: 100%|██████████| 67/67 [00:06<00:00, 10.45it/s, accuracy=0.381, cost=3.91]

time taken: 101.39822745323181
epoch: 13, training loss: 0.024281, training acc: 0.993200, valid loss: 2.515767, valid acc: 0.598308

break epoch:14

